### Import rdflib and create functions

* g is the ontology graph where the infered relations are added to
* Serialize is a function that transforms the ontology into a turle format
* Save is a function that can save the ontology as a turtle file
* Load is a function that loads the ontology into this file
* Shorten is a function that shows the prefix + subject, property or object    vb. kinship1:has_mother
* Relation shows only the subject, property or object

In [1]:
from rdflib import Graph, RDF, RDFS, Namespace, Literal, URIRef 
import pandas as pd

pd.set_option('display.max_rows', 999)


df = pd.DataFrame()
g = Graph()

KIN = Namespace('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#')
g.bind('kinship1',KIN)

def serialize():
    print(g.serialize(format='turtle').decode("utf-8"))

def save(filename):
    with open(filename, 'wb') as f:
        g.serialize(f, format='turtle')
        
def load(filename):
    with open(filename, 'r') as f:
        g.load(f, format='turtle') 
    
def shorten(uri):
    if isinstance(uri, Literal):
        return uri    
    components = g.namespace_manager.compute_qname(uri)
    return '%s:%s'%(components[0], components[2])

def relation(uri):
    if isinstance(uri, Literal):
        return uri    
    components = g.namespace_manager.compute_qname(uri)
    return '%s'%(components[2])

## Create counter that counts the amount of newly created relations in this ontology.

In [2]:
new_relations_counter = 0

### Load in the ontology and print in turtle format

In [3]:
load('kinship.ttl')
serialize()

@prefix : <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1> .
@prefix kinship1: <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

: a owl:Ontology .

kinship1:Andrea_Waltham a kinship1:Human,
        kinship1:woman,
        owl:NamedIndividual ;
    kinship1:has_Daughter kinship1:Emily_Waltham .

kinship1:Brother_in_law a owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Brother in law"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    rdfs:subPropertyOf kinship1:In_law ;
    owl:propertyDisjointWith kinship1:Sister_in_law .

kinship1:Cousins_husband a owl:AsymmetricProperty,
        owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Cousins husband"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    owl:propertyDisjointWith kinship1:Cousins_wife .

kinship1:Daugther_in_law

### Create empty lists and values that have to be filled later.

In [4]:
parent  = []
parent2 = []
father  = []
mother  = []
child   = []
child2  = []
son     = []
daughter = []
grandchild = []
grandchild2 = []
grandparent = []
grandparent2 = []
inv1    = []
inv2    = []
disj1   = []
disj2   = []
couple  = []
couple2 = []
marriage = []
marriage2 = []
marriage3 = []
marriage4 = []
marriage5 = []
marriage6 = []
marriage7 = []
marriage8 = []
cousin1 = []
cousin2 = []
sibling1 = []
sibling2 = []

### Create a dataframe that later shows the inferred gender relations and dataframe that shows the inferred relations.

In [5]:
df = pd.DataFrame(columns = ["Relation", "Subject", "Property", "Object"])
df2 = pd.DataFrame(columns = ["Subject", "Property", "Object"])
df5 = pd.DataFrame(columns = ["Subject", "Property", "Object"])
# df8 = pd.DataFrame(columns = ["Subject", "Property", "Object"])
df_sibling =  pd.DataFrame(columns = ["Subject", "Object"])
# df25 = pd.DataFrame(columns = ["Subject", "Property", "Object"])

 ### Fill the lists with the right values and add gender to individuals
 The known relations will be added seperatly in lists and the genders of the individuals will be added to the ontology by determining what their relation is with other individuals. For example, if someone is a mother she will automatically also be a woman.

In [6]:
for s,p,o in g:
# PARENT RELATIONS
    if shorten(p) == "kinship1:has_parent" or shorten(p) == "kinship1:has_Father" or shorten(p) == "kinship1:has_Mother":
        parent.append(o)
        child.append(s) 
    if shorten(p) == "kinship1:has_Father":
        df = df.append({'Relation': "Father", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Mother":
        df = df.append({'Relation': "Mother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# GRANDPARENT RELATIONS
    if shorten(p) == "kinship1:has_Grandparent" or shorten(p) == "kinship1:has_Grandfather " or shorten(p) == "kinship1:has_Grandmother":
        grandparent.append(o)
        grandchild.append(s)    
    if shorten(p) == "kinship1:has_Grandfather":
        df = df.append({'Relation': "Grandfather", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Grandmother":
        df = df.append({'Relation': "Grandmother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# CHILD RELATIONS       
    if shorten(p) == "kinship1:has_Child" or shorten(p) == "kinship1:has_Son" or shorten(p) == "kinship1:has_Dauther": #typo in ontology daughter
        child2.append(o)
        parent2.append(s)    
    if shorten(p) == "kinship1:has_Son":
        df = df.append({'Relation': "Son", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Dauther":
        df = df.append({'Relation': "Daughter", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1

# GRANDCHILD RELATIONS       
    if shorten(p) == "kinship1:has_Grandchild" or shorten(p) == "kinship1:has_Grandson" or shorten(p) == "kinship1:has_Granddaughter": 
        grandchild2.append(o)
        grandparent2.append(s)   
    if shorten(p) == "kinship1:has_Grandson":
        df = df.append({'Relation': "Grandson", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Granddaughter":
        df = df.append({'Relation': "Granddaughter", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# SIBLING RELATIONS
    if shorten(p) == "kinship1:has_Sibling" or  shorten(p) == "kinship1:has_Sister" or shorten(p) == "kinship1:has_Brother": 
        sibling1.append(o)
        sibling2.append(s)
    if shorten(p) == "kinship1:has_Brother":
        df = df.append({'Relation': "Brother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Sister":
        df = df.append({'Relation': "Sister", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1

# COUSIN RELATIONS
    if shorten(p) == "kinship1:has_Cousin":
        cousin1.append(o)
        cousin2.append(s)

#RELATIONSHIPS 
    if shorten(p) == "kinship1:has_Husband" or shorten(p) == "kinship1:has_Wife":
        couple.append(o)
        couple2.append(s)
    if shorten(p) == "kinship1:has_Husband":
        marriage.append(o)
        marriage2.append(s)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
        df = df.append({'Relation': "Husband", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Wife":
        marriage3.append(o)
        marriage4.append(s)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        df = df.append({'Relation': "Wife", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Ex_husband":
        marriage5.append(o)
        marriage6.append(s)
        g.add((o, RDFS.domain, KIN.man)) 
        new_relations_counter +=1
        df = df.append({'Relation': "Ex_Husband", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Ex_wife":
        marriage7.append(o)
        marriage8.append(s)
        g.add((o, RDFS.domain, KIN.woman))  
        new_relations_counter +=1
        df = df.append({'Relation': "Ex_Wife", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)

### Print a table with the inferred gender relations that are added to the ontology

In [7]:
df = df.drop_duplicates().sort_values(by=['Relation'])
df.reset_index(drop=True)

,Relation,Subject,Property,Object
0,Brother,Frank_Buffay,domain,man
1,Ex_Husband,Duncan,domain,man
2,Ex_Wife,Emily_Waltham,domain,woman
3,Ex_Wife,Carol_Willick,domain,woman
4,Father,Charles_Bing,domain,man
5,Father,Jack_Geller,domain,man
6,Father,Leonard_Green,domain,man
7,Father,Ross,domain,man
8,Grandmother,Frances,domain,woman
9,Husband,Mike_Hannigan,domain,man


In [8]:
serialize() 

@prefix : <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1> .
@prefix kinship1: <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

: a owl:Ontology .

kinship1:Andrea_Waltham a kinship1:Human,
        kinship1:woman,
        owl:NamedIndividual ;
    kinship1:has_Daughter kinship1:Emily_Waltham .

kinship1:Brother_in_law a owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Brother in law"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    rdfs:subPropertyOf kinship1:In_law ;
    owl:propertyDisjointWith kinship1:Sister_in_law .

kinship1:Cousins_husband a owl:AsymmetricProperty,
        owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Cousins husband"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    owl:propertyDisjointWith kinship1:Cousins_wife .

kinship1:Daugther_in_law

### Extract all the necessary properties

In [9]:
for s,p,o in g:
    if shorten(p) == "owl:inverseOf":
        inv1.append(o)
        inv2.append(s)
        if shorten(s) == "kinship1:has_Child":
            child_property = s 
        if shorten(o) == "kinship1:has_parent":
            parent_property = o
        if shorten(s) == "kinship1:has_Grandchild":
            grandchild_property = s
        if shorten(o) == "kinship1:has_Grandparent":
            grandparent_property = o
        
    if shorten(p) == "owl:propertyDisjointWith":
        disj1.append(o)
        disj2.append(s)
        if shorten(s) == "kinship1:has_Husband":
            husband_property = s
        if shorten(o) == "kinship1:has_Wife":
            wife_property = o
        if shorten(s) == "kinship1:has_Ex_husband":
            ex_husband_property = s
        if shorten(o) == "kinship1:has_Ex_wife":
            ex_wife_property = o
            
    if shorten(o) == "owl:SymmetricProperty":
        symprop = shorten(o)
        if shorten(s) ==  "kinship1:has_Cousin":
            cousin_property = s
        if shorten(s) == "kinship1:has_Sibling":
            sibling_property = s
            

    if shorten(o) == "owl:IrreflexiveProperty":
        if shorten(s) == "kinship1:has_Partner":
            partner_property = s
        if shorten(s) == "kinship1:has_Father":
            father_property = s
        if shorten(s) == "kinship1:has_Mother":
            mother_property = s
        if shorten(s) ==  "kinship1:has_Son"  :
            son_property = s
        if shorten(s) == "kinship1:has_Daughter":
            daughter_property = s
        if shorten(s) == "kinship1:has_Grandson":
            grandson_property = s
        if shorten(s) == "kinship1:has_Granddaughter":
            granddaughter_property = s
        if shorten(s) == "kinship1:has_Grandfather":
            grandfather_property = s
        if shorten(s) == "kinship1:has_Grandmother":
            grandmother_property = s
        if shorten(s) == "kinship1:has_Brother":
            brother_property = s
        if shorten(s) == "kinship1:has_Sister":
            sister_property = s
        if shorten(s) == "kinship1:has_twin":
            twin_property = s

Print the properties to show what they look like.

In [10]:
print("child_property", child_property)
print("parent_property", parent_property)
print("grandchild_property", grandchild_property)
print("grandparent_property", grandparent_property)
print("husband_property", husband_property)
print("wife_property", wife_property)
print("ex_husband_property", ex_husband_property)
print("ex_wife_property", ex_wife_property)
print("cousin_property", cousin_property)
print("partner_property", partner_property)
print("father_property", father_property)
print("mother_property", mother_property)
print("son_property", son_property)
print("daughter_property", daughter_property)
print("grandson_property", grandson_property)
print("granddaughter_property", granddaughter_property)
print("grandfather_property", grandfather_property)
print("grandmother_property", grandmother_property)
print("brother_property", brother_property)
print("sister_property", sister_property)
print("twin_property", twin_property)
print("sibling_property", sibling_property)


child_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Child
parent_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_parent
grandchild_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Grandchild
grandparent_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Grandparent
husband_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Husband
wife_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Wife
ex_husband_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Ex_husband
ex_wife_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Ex_wife
cousin_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Cousin
partner_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Partner
father_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Father
mother

### Create functions to iterate through the lists and add the reversed relations with the right property to the ontology.
All the added relations are also added to a dataframe.

In [11]:
def create_inferrence(new_subject, new_property, new_object, ignore_index=True):
    global df2
    global new_relations_counter
    for i in range(len(new_subject)):
        g.add((new_subject[i], new_property, new_object[i]))
        new_relations_counter +=1
        data = [[ relation(new_subject[i]), relation(new_property), relation(new_object[i])]]
        df3 = pd.DataFrame(data, columns = ['Subject', 'Property', "Object"])
        df2 = df2.append(df3)

In [12]:
create_inferrence(parent, child_property, child)
create_inferrence(child2, parent_property, parent2)
create_inferrence(grandparent, grandchild_property, grandchild)
create_inferrence(grandchild2, grandparent_property, grandparent2)

create_inferrence(couple, partner_property, couple2)
create_inferrence(marriage, wife_property, marriage2)
create_inferrence(marriage3, husband_property, marriage4)
create_inferrence(marriage5, ex_wife_property, marriage6)
create_inferrence(marriage7, ex_husband_property, marriage8)

create_inferrence(cousin1, cousin_property, cousin2)

In [13]:
df2.reset_index(drop=True)

,Subject,Property,Object
0,Phoebe_Abbott,has_Child,Phoebe
1,Charles_Bing,has_Child,Chandler_Bing
2,Jack_Geller,has_Child,Ross
3,Leonard_Green,has_Child,Rachel
4,Sandra_Green,has_Child,Rachel
5,Joseph_Tribbiani,has_Child,Joey_Tribbiani
6,Ross,has_Child,Ben
7,Judy_Geller,has_Child,Ross
8,Theodore_Hannigan,has_Child,Mike_Hannigan
9,Bitsy_Hannigan,has_Child,Mike_Hannigan


In [14]:
serialize() 

@prefix : <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1> .
@prefix kinship1: <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

: a owl:Ontology .

kinship1:Andrea_Waltham a kinship1:Human,
        kinship1:woman,
        owl:NamedIndividual ;
    kinship1:has_Daughter kinship1:Emily_Waltham .

kinship1:Brother_in_law a owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Brother in law"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    rdfs:subPropertyOf kinship1:In_law ;
    owl:propertyDisjointWith kinship1:Sister_in_law .

kinship1:Cousins_husband a owl:AsymmetricProperty,
        owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Cousins husband"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    owl:propertyDisjointWith kinship1:Cousins_wife .

kinship1:Daugther_in_law

## Extract all the man and woman. And if someone participates in a relation that is not dependent on gender, they will be added to the appropriate list. 

In [15]:
woman_list = []
man_list = []
has_parent_relation = []
has_child_relation = []
has_grandparent_relation = []
has_grandchild_relation = []
has_sibling_relation = []

for s,p,o in g:
    if shorten(o) ==  "kinship1:woman":
        woman_list.append(s)
    if shorten(o) ==  "kinship1:man":
        man_list.append(s)
    if shorten(p) == "kinship1:has_parent":# or shorten(p) == "kinship1:has_Father" or shorten(p) == "kinship1:has_Mother":
        has_parent_relation.append((o, s))
    if shorten(p) == "kinship1:has_Child":# or shorten(p) == "kinship1:has_Son" or shorten(p) == "kinship1:has_Dauther":
        has_child_relation.append((o, s))
    if shorten(p) == "kinship1:has_Grandparent":# or shorten(p) == "kinship1:has_Grandfather " or shorten(p) == "kinship1:has_Grandmother":
        has_grandparent_relation.append((o, s))
    if shorten(p) == "kinship1:has_Grandchild": #or shorten(p) == "kinship1:has_Grandson" or shorten(p) == "kinship1:has_Granddaughter":
        has_grandchild_relation.append((o, s))
    if shorten(p) == "kinship1:has_Sibling":
        has_sibling_relation.append((o,s))

## Add relations that are gender based.

In [16]:
mothers = set(woman_list) & set([x[0] for x in has_parent_relation])
fathers = set(man_list) & set([x[0] for x in has_parent_relation])
daughters = set(woman_list) & set([x[0] for x in has_child_relation])
sons = set(man_list) & set([x[0] for x in has_child_relation])

sisters = set(woman_list) & set([x[0] for x in has_sibling_relation])
brothers = set(man_list) & set([x[0] for x in has_sibling_relation])

grandmothers = set(woman_list) & set([x[0] for x in has_grandparent_relation])
grandfathers = set(man_list) & set([x[0] for x in has_grandparent_relation])
granddaughters = set(woman_list) & set([x[0] for x in has_grandchild_relation])
grandsons = set(man_list) & set([x[0] for x in has_grandchild_relation])
# grandmothers, grandfathers, granddaughters, grandsons, sisters, brothers

### Create functions to iterate through the lists and add the reversed relations with the right property to the ontology.
All the added relations are also added to a dataframe.

In [17]:
def create_gender_based_inferrence(subject_list, new_property, relationship, ignore_index=True):
    global df5  
    global new_relations_counter
    for x in subject_list:
        if x[0] in relationship:
            g.add((x[1], new_property, x[0]))
            new_relations_counter +=1
            data = [[ relation(x[1]), relation(new_property), relation(x[0])]]
            df3 = pd.DataFrame(data, columns = ['Subject', 'Property', "Object"])
            df5 = df5.append(df3)

In [18]:
create_gender_based_inferrence(has_parent_relation, mother_property, mothers)
create_gender_based_inferrence(has_parent_relation, father_property, fathers)
create_gender_based_inferrence(has_child_relation, daughter_property, daughters)
create_gender_based_inferrence(has_child_relation, son_property, sons)
create_gender_based_inferrence(has_grandparent_relation, grandmother_property, grandmothers)
create_gender_based_inferrence(has_grandparent_relation, grandfather_property, grandfathers)
create_gender_based_inferrence(has_grandchild_relation, granddaughter_property, granddaughters)
create_gender_based_inferrence(has_grandchild_relation, grandson_property, grandsons)
create_gender_based_inferrence(has_sibling_relation, sister_property, sisters)
create_gender_based_inferrence(has_sibling_relation, brother_property, brothers)

### Print a dataframe with all the inferred relations. 
These relations are all gender based, so father instead of parent ect.

In [19]:
df5 = df5.sort_values(by=['Property'])
df5.reset_index(drop=True)

,Subject,Property,Object
0,Jack_Geller,has_Daughter,Monica
1,Judy_Geller,has_Daughter,Monica
2,Phoebe_Abbott,has_Daughter,Phoebe
3,Sandra_Green,has_Daughter,Rachel
4,Leonard_Green,has_Daughter,Rachel
5,Joey_Tribbiani,has_Father,Joseph_Tribbiani
6,Mike_Hannigan,has_Father,Theodore_Hannigan
7,Ben,has_Father,Ross
8,Frances,has_Granddaughter,Phoebe
9,Ben,has_Mother,Carol_Willick


In [20]:
serialize() 

@prefix : <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1> .
@prefix kinship1: <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

: a owl:Ontology .

kinship1:Andrea_Waltham a kinship1:Human,
        kinship1:woman,
        owl:NamedIndividual ;
    kinship1:has_Daughter kinship1:Emily_Waltham .

kinship1:Brother_in_law a owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Brother in law"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    rdfs:subPropertyOf kinship1:In_law ;
    owl:propertyDisjointWith kinship1:Sister_in_law .

kinship1:Cousins_husband a owl:AsymmetricProperty,
        owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Cousins husband"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    owl:propertyDisjointWith kinship1:Cousins_wife .

kinship1:Daugther_in_law

### Add another round of gender assigning, because there may be some individuals that can now be assigned a gender.

In [21]:
for s,p,o in g:
# PARENT RELATIONS
    if shorten(p) == "kinship1:has_Father":
        df = df.append({'Relation': "Father", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Mother":
        df = df.append({'Relation': "Mother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# GRANDPARENT RELATIONS
    if shorten(p) == "kinship1:has_Grandfather":
        df = df.append({'Relation': "Grandfather", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Grandmother":
        df = df.append({'Relation': "Grandmother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# CHILD RELATIONS       
    if shorten(p) == "kinship1:has_Son":
        df = df.append({'Relation': "Son", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Dauther":
        df = df.append({'Relation': "Daughter", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1

# GRANDCHILD RELATIONS        
    if shorten(p) == "kinship1:has_Grandson":
        df = df.append({'Relation': "Grandson", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Granddaughter":
        df = df.append({'Relation': "Granddaughter", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1

# SIBLING RELATIONS
    if shorten(p) == "kinship1:has_Brother":
        df = df.append({'Relation': "Brother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Sister":
        df = df.append({'Relation': "Sister", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
#RELATIONSHIPS 
    if shorten(p) == "kinship1:has_Husband":
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
        df = df.append({'Relation': "Husband", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Wife":
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        df = df.append({'Relation': "Wife", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Ex_husband":
        g.add((o, RDFS.domain, KIN.man)) 
        new_relations_counter +=1
        df = df.append({'Relation': "Ex_Husband", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Ex_wife":
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        df = df.append({'Relation': "Ex_Wife", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)

In [22]:
df = df.sort_values(by=['Subject'])
df = df.drop_duplicates()
df = df.drop(['Property', 'Object'], axis=1)
columns_titles = ["Subject","Relation"]
df = df.reindex(columns=columns_titles)
print("\n","A dataframe of all the individuals with the relations in which they participate.")
df.reset_index(drop=True)


 A dataframe of all the individuals with the relations in which they participate.


,Subject,Relation
0,Alice_Knight,Wife
1,Amy_Green,Sister
2,Ben,Son
3,Bitsy_Hannigan,Mother
4,Carol_Willick,Mother
5,Carol_Willick,Ex_Wife
6,Carol_Willick,Husband
7,Chandler_Bing,Son
8,Charles_Bing,Father
9,Cookie,Sister


### Add sibling relations

In [23]:
sibling_infer_relation_list = []
sibling_infer_relation_list_show = []
name_list = []

for s,p,o in g:
    if shorten(p) == "kinship1:has_Sibling" or relation(p) == "has_Sister" or relation(p) == "has_Brother"  or relation(p) == "has_twin":
        sibling_infer_relation_list_show.append((relation(s), relation(o)))
        sibling_infer_relation_list.append((s, o))
        name_list.append(relation(s))
        name_list.append(relation(o))

Show all the sibling relations that exist at the moment.

In [24]:
sibling_infer_relation_list_show

[('Joey_Tribbiani', 'Mary_Therese_Tribbiani'),
 ('Joey_Tribbiani', 'Dina'),
 ('Amy_Green', 'Rachel'),
 ('Ross', 'Monica'),
 ('Mary_Angela_Tribbiani', 'Dina'),
 ('Joey_Tribbiani', 'Tina'),
 ('Joey_Tribbiani', 'Mary_Angela_Tribbiani'),
 ('Ross', 'Monica'),
 ('Phoebe', 'Frank_Buffay'),
 ('Amy_Green', 'Jill_Green'),
 ('Joey_Tribbiani', 'Veronica'),
 ('Rachel', 'Jill_Green'),
 ('Rachel', 'Amy_Green'),
 ('Phoebe', 'Ursula_Buffay'),
 ('Joey_Tribbiani', 'Cookie'),
 ('Joey_Tribbiani', 'Gina')]

In [25]:
name_data = pd.DataFrame(name_list, columns = ["Name"])
final_name_list = name_data.drop_duplicates().reset_index(drop=True)
print("\n", "These are all the individuals that participate in a sibling relation:")
final_name_list


 These are all the individuals that participate in a sibling relation:


,Name
0,Joey_Tribbiani
1,Mary_Therese_Tribbiani
2,Dina
3,Amy_Green
4,Rachel
5,Ross
6,Monica
7,Mary_Angela_Tribbiani
8,Tina
9,Phoebe


### Put all the individuals that are siblings of each other in a set together and put those sets in a list. 

In [26]:
def make_word_classes(synom):
    # List of classes
    word_classes = []
    for w1, w2 in synom:
        # Go through existing classes
        for wcls in word_classes:
            # If one of the words is already in the current class
            if w1 in wcls or w2 in wcls:
                # Add both words and continue to next pair of words
                wcls.add(w1)
                wcls.add(w2)
                break
        else:  # Note this else goes with the for loop, not the if block
            # If there was no matching class, add a new one
            word_classes.append({w1, w2})
    return word_classes

synom = sibling_infer_relation_list
show = sibling_infer_relation_list_show
word_classes = make_word_classes(synom)
print("This is what it looks like right now:","\n", make_word_classes(show))

This is what it looks like right now: 
 [{'Mary_Therese_Tribbiani', 'Dina', 'Veronica', 'Tina', 'Cookie', 'Gina', 'Joey_Tribbiani', 'Mary_Angela_Tribbiani'}, {'Rachel', 'Amy_Green', 'Jill_Green'}, {'Monica', 'Ross'}, {'Frank_Buffay', 'Ursula_Buffay', 'Phoebe'}]


### Iterate through the list with the sets and add a sibling relation between all the individuals of a set except when the individuals are the same.

In [27]:
vowels = word_classes
vowels_iter = iter(word_classes)

while True: 
    try: 
        for i in word_classes:                 #x en i printen hetzelfde
            x = next(vowels_iter)
            for p in i:                         # p is 1 persoon
                for y in i:                 #p blijft 1 ronde dezelfde persoon en y itereerd tijdens die ronde door de personen
                    if p == y:
                        pass
                    else:
                        g.add((p, sibling_property,y))
                        new_relations_counter +=1
          
    except StopIteration: 
        # if StopIteration is raised, break from loop 
        break


### Add the sibling relations to the dataframe to show all the inferences.

In [28]:
woman_list = []
man_list = []
has_sibling_relation = []
has_twin_relation=[]

for s,p,o in g:
    if shorten(o) ==  "kinship1:woman":
        woman_list.append(s)
    if shorten(o) ==  "kinship1:man":
        man_list.append(s)
    if shorten(p) == "kinship1:has_Sibling":
        has_sibling_relation.append((o,s))
    if shorten(p) == "kinship1:has_twin":
        has_twin_relation.append((s,o))
sisters = set(woman_list) & set([x[0] for x in has_sibling_relation])
brothers = set(man_list) & set([x[0] for x in has_sibling_relation])
twin_sisters = set(woman_list) & set([x[0] for x in has_twin_relation])
twin_brothers = set(man_list) & set([x[0] for x in has_twin_relation])

create_gender_based_inferrence(has_sibling_relation, sister_property, sisters)
create_gender_based_inferrence(has_sibling_relation, brother_property, brothers)
create_gender_based_inferrence(has_twin_relation, twin_property, twin_brothers)
create_gender_based_inferrence(has_twin_relation, twin_property, twin_sisters)

In [29]:
df5 = df5.drop_duplicates().sort_values(by=['Property'])
df5.reset_index(drop=True)

,Subject,Property,Object
0,Mary_Therese_Tribbiani,has_Brother,Joey_Tribbiani
1,Tina,has_Brother,Joey_Tribbiani
2,Dina,has_Brother,Joey_Tribbiani
3,Mary_Angela_Tribbiani,has_Brother,Joey_Tribbiani
4,Gina,has_Brother,Joey_Tribbiani
5,Phoebe,has_Brother,Frank_Buffay
6,Veronica,has_Brother,Joey_Tribbiani
7,Cookie,has_Brother,Joey_Tribbiani
8,Monica,has_Brother,Ross
9,Ursula_Buffay,has_Brother,Frank_Buffay


# Show final ontology

In [30]:
serialize()

@prefix : <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1> .
@prefix kinship1: <http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

: a owl:Ontology .

kinship1:Andrea_Waltham a kinship1:Human,
        kinship1:woman,
        owl:NamedIndividual ;
    kinship1:has_Daughter kinship1:Emily_Waltham .

kinship1:Brother_in_law a owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Brother in law"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    rdfs:subPropertyOf kinship1:In_law ;
    owl:propertyDisjointWith kinship1:Sister_in_law .

kinship1:Cousins_husband a owl:AsymmetricProperty,
        owl:IrreflexiveProperty,
        owl:ObjectProperty ;
    rdfs:label "Cousins husband"@en ;
    rdfs:domain kinship1:man ;
    rdfs:range kinship1:Human ;
    owl:propertyDisjointWith kinship1:Cousins_wife .

kinship1:Daugther_in_law

In [31]:
save('new_kinship1_ontology.ttl')

# Show the result of the counter of the newly added relations to the ontology


In [32]:
new_relations_counter

281